In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

In [3]:
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chat_models.base import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.vectorstores import VectorStore
from langchain.memory.chat_message_histories import SQLChatMessageHistory

from backend.config_renderer import get_config
from backend.rag_components.embedding import get_embedding_model
from backend.rag_components.llm import get_llm_model
from backend.rag_components.vector_store import get_vector_store
import frontend.lib.auth as auth

In [4]:
def get_answer_chain(llm, docsearch: VectorStore, memory) -> ConversationalRetrievalChain:
    """Returns an instance of ConversationalRetrievalChain based on the provided parameters."""
    template = """Given the conversation history and the following question, can you rephrase the user's question in its original language so that it is self-sufficient. Make sure to avoid the use of unclear pronouns.

Chat history :
{chat_history}
Question : {question}

Rephrased question :
"""
    condense_question_prompt = PromptTemplate.from_template(template)
    condense_question_chain = LLMChain(
        llm=llm,
        prompt=condense_question_prompt,
    )

    messages = [
        SystemMessage(
            content=(
                """As a chatbot assistant, your mission is to respond to user inquiries in a precise and concise manner based on the documents provided as input. It is essential to respond in the same language in which the question was asked. Responses must be written in a professional style and must demonstrate great attention to detail."""
            )
        ),
        HumanMessage(content="Respond to the question taking into account the following context."),
        HumanMessagePromptTemplate.from_template("{context}"),
        HumanMessagePromptTemplate.from_template("Question: {question}"),
    ]
    system_prompt = ChatPromptTemplate(messages=messages)
    qa_chain = LLMChain(
        llm=llm,
        prompt=system_prompt,
    )

    doc_prompt = PromptTemplate(
        template="Content: {page_content}\nSource: {source}",
        input_variables=["page_content", "source"],
    )

    final_qa_chain = StuffDocumentsChain(
        llm_chain=qa_chain,
        document_variable_name="context",
        document_prompt=doc_prompt,
    )

    return ConversationalRetrievalChain(
        question_generator=condense_question_chain,
        retriever=docsearch.as_retriever(search_kwargs={"k": 10}),
        memory=memory,
        combine_docs_chain=final_qa_chain,
        verbose=True,
    )


In [5]:
config = get_config()
llm = get_llm_model(config)
embeddings = get_embedding_model(config)
vector_store = get_vector_store(embeddings, config)

In [6]:
username = "slauzeral"
password = "test"

In [8]:
success = auth.sign_up(username, password)
if success:
    token = auth.get_token(username, password)
    session = auth.create_session()
    auth_session = auth.authenticate_session(session, token)

response = auth_session.post("/chat/new")
chat_id = response.json()["chat_id"]

In [20]:
from datetime import datetime
from uuid import uuid4
from typing import Any

from langchain.memory.chat_message_histories.sql import DefaultMessageConverter
from langchain.schema import AIMessage, BaseMessage, HumanMessage, SystemMessage
from sqlalchemy import Column, DateTime, Integer, Text
from sqlalchemy.orm import declarative_base
from langchain.schema.messages import BaseMessage, _message_to_dict, messages_from_dict
import json

Base = declarative_base()

class CustomMessage(Base):
    __tablename__ = "message_test"

    id = Column(Text, primary_key=True,  default=lambda: str(uuid4())) # default=lambda: str(uuid4())
    timestamp = Column(DateTime)
    chat_id = Column(Text)
    sender = Column(Text)
    content = Column(Text)
    message = Column(Text)


class CustomMessageConverter(DefaultMessageConverter):

    def to_sql_model(self, message: BaseMessage, session_id: str) -> Any:
        sub_message = json.loads(message.content)
        return CustomMessage(
            id = sub_message["id"],
            timestamp = datetime.strptime(sub_message["timestamp"], "%Y-%m-%d %H:%M:%S.%f"),
            chat_id = session_id,
            sender = message.type,
            content = sub_message["content"],
            message = json.dumps(_message_to_dict(message)),
        )

    def get_sql_model_class(self) -> Any:
        return CustomMessage


In [21]:
chat_message_history = SQLChatMessageHistory(
    session_id=chat_id,
    connection_string="sqlite:////Users/sarah.lauzeral/Library/CloudStorage/GoogleDrive-sarah.lauzeral@artefact.com/Mon Drive/internal_projects/skaff-rag-accelerator/database/database.sqlite",
    table_name="message_test",
    session_id_field_name="chat_id",
    custom_message_converter=CustomMessageConverter(table_name="message_test"),
)

In [22]:
chat_message_history.add_ai_message(json.dumps({"content":"Hi", "timestamp":f"{datetime.utcnow()}", "id":"764528762"}))

In [23]:
chat_message_history.messages

[AIMessage(content='{"content": "Hi", "timestamp": "2023-12-20 16:26:23.672506", "id": "764528762"}')]